In [1]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)
import optuna

import pickle
from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error

import statistics as st
import lightgbm as lgbm
from scipy.stats import norm

In [2]:
sys.path.append('../../')
import src.utils as utils

## Param

In [3]:
EXP_NUM = 76
NFOLDS = 5
SEED = 42

In [4]:
# def set_seed(seed: int = 42):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)  # type: ignore
#     torch.backends.cudnn.deterministic = True  # type: ignore
#     torch.backends.cudnn.benchmark = False  # type: ignore
# set_seed(SEED)

## Dir

In [5]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [6]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv')
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')
salaries = pd.read_csv(MAIN_DATA_DIR / 'mlbSalaries.csv')
teams = pd.read_csv(MAIN_DATA_DIR / 'teams.csv')

standings = pd.read_csv(TRAIN_DIR / 'standings_train.csv')
playerTwitterFollowers = pd.read_csv(TRAIN_DIR / 'playerTwitterFollowers_train.csv')

# events = pd.read_csv(TRAIN_DIR / 'events_train.csv')
# events = events.groupby(['gameDate']).sum().reset_index()


In [7]:
train_elements_dict = {"players":players, 
                       "rosters":rosters, 
                       "targets":targets, 
                       "scores":scores, 
                       "seasons":seasons, 
                       "teams":teams, 
                       "standings":standings}

In [8]:
def map_team_name(name):
    names = name.split('-')
    result = ''
    for n in names:
        if n == 'st':
            n = 'st.'
        result += f' {n.capitalize()}'

    return result[1:]

In [9]:
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]
    corr_ary = []

    for i in range(n):
        for j in range(i):
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])

    return corr_ary

In [10]:
def calc_probs(pid,df,temp):
    to_append=[pid,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    targets=['target1','target2','target3','target4']
    z=1
    for target in targets:
        target_prob = temp[target].tolist()
        mean = np.mean(target_prob)
        std = np.std(target_prob)
        median = st.median(target_prob)
        distribution = norm(mean, std)
        min_weight = min(target_prob)
        max_weight = max(target_prob)
        values = list(np.linspace(min_weight, max_weight))
        probabilities = [distribution.pdf(v) for v in values]
        max_value = max(probabilities)
        max_index = probabilities.index(max_value)
        to_append[z]=mean
        to_append[z+1]=median
        to_append[z+2]=std
        to_append[z+3]=min_weight
        to_append[z+4]=max_weight
        to_append[z+5]=temp[target].skew()
        to_append[z+6]=temp[target].kurt()

        z=z+7
    corr_ = calc_corr(temp[['target1', 'target2', 'target3', 'target4']])
    to_append[z:] = corr_  
    df_length = len(df)
    df.loc[df_length] = to_append
    return df

In [11]:
def count_consecutive_items_n_cols(df, col_name_list, output_col):
    cum_sum_list = [
        (df[col_name] != df[col_name].shift(1)).cumsum().tolist() for col_name in col_name_list
    ]
    df[output_col] = df.groupby(
        ["_".join(map(str, x)) for x in zip(*cum_sum_list)]
    ).cumcount() + 1
    return df

In [12]:
def extract_season(date_raw, season_start_end):
    idxes = 0
    for raw in season_start_end.iloc():
        idx_ = ((date_raw >= raw.iloc[0]) & (date_raw <= raw.iloc[1])) * 1
        idxes += idx_
    return idxes

In [13]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     oof_pred = np.clip(oof_pred, 0, 100)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score

In [14]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, verbose=100):
    def opt(trial):
        params = {
                'random_state': SEED,
                'objective':'mae',
                'n_estimators': 10000,
                'learning_rate': 0.1,
                'max_depth': trial.suggest_int('max_depth', 1, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
                'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
                'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 20),
                'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        }

        model_opt = lgbm.LGBMRegressor(**params)

        model_opt.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)
        oof_pred = model_opt.predict(x_valid)
        oof_pred = np.clip(oof_pred, 0, 100)
        score = mean_absolute_error(oof_pred, y_valid)
        return -score
    return opt

In [15]:
def my_timeseries_fold(train):
    
    tr_idxs = []
    val_idxs = []
    
    tr_idx = (train['date'].astype(int) < 20200801)
    val_idx = (train['date'].astype(int) >= 20200801) & (train['date'].astype(int) < 20200901)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20200901)
    val_idx = (train['date'].astype(int) >= 20200901) & (train['date'].astype(int) < 20201001)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20201001)
    val_idx = (train['date'].astype(int) >= 20201001) & (train['date'].astype(int) < 20201028)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20210228)
    val_idx = (train['date'].astype(int) >= 20210228) & (train['date'].astype(int) < 20210401)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20210401)
    val_idx = ~tr_idx
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)
    
    return tr_idxs, val_idxs

In [16]:
class Rt4kaidoTrain:
    def __init__(self, usetimelinefeature=False):
        
        self.usetimelinefeature = usetimelinefeature
        self.targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
        self.players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'mlbDebutDate', 'DebutAge', 'heightInches', 'weight', 'playerForTestSetAndFuturePreds']
        self.rosters_cols = ['playerId', 'teamId', 'status', 'date']
        self.salaries_cols = ['teamId', 'salary', 'year']
        self.standings_cols = ['teamId', 'wildCardRank', 'sportGamesBack', 'date']
        self.transactions_cols = ['playerId', 'transaction_flag', 'date']
        self.stat_cols = ["playerId", "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
                        "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
                        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
                        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt",
                        'tgt1_2_corr', 'tgt1_3_corr', 'tgt2_3_corr', 'tgt1_4_corr', 'tgt2_4_corr', 'tgt3_4_corr']

        self.scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances', 'date']

        self.feature_cols1 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 

        self.feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank'] 

        self.feature_cols3 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'diffmlbDebutDateflag', 'sincemlbDebutDateflag'] 

        self.feature_cols4 = ['week_day', 'annual_day', 'month', 'label_playerId', 'label_primaryPositionName', 'label_teamId', 'label_birthCity',
                        'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'diffmlbDebutDateflag', 'sincemlbDebutDateflag'] 
        
        
        # lightgbm
        self.params1 = {'objective':'mae',
                       'reg_alpha': 0.14947461820098767, 
                       'reg_lambda': 0.10185644384043743, 
                       'n_estimators': 3633, 
                       'learning_rate': 0.08046301304430488, 
                       'num_leaves': 674, 
                       'feature_fraction': 0.9101240539122566, 
                       'bagging_fraction': 0.9884451442950513, 
                       'bagging_freq': 8, 
                       'min_child_samples': 51}


        self.params2 = {'objective':'mae',
                       'reg_alpha': 0.1,
                       'reg_lambda': 0.1, 
                       'n_estimators': 80,
                       'learning_rate': 0.1,
                       'random_state': 42,
                       "num_leaves": 22}



        self.params3 = {'objective':'mae',
                       'reg_alpha': 0.1,
                       'reg_lambda': 0.1, 
                       'n_estimators': 10000,
                       'learning_rate': 0.1,
                       'random_state': 42,
                       "num_leaves": 100}

        self.params4 = {'objective':'mae',
                       'reg_alpha': 0.016468100279441976, 
                       'reg_lambda': 0.09128335764019105, 
                       'n_estimators': 9868, 
                       'learning_rate': 0.10528150510326864, 
                       'num_leaves': 157, 
                       'feature_fraction': 0.5419185713426886, 
                       'bagging_fraction': 0.2637405128936662, 
                       'bagging_freq': 19, 
                       'min_child_samples': 71}

    def make_feature(self, train_elements_dict):

        players = train_elements_dict['players']
        rosters = train_elements_dict['rosters']
        targets = train_elements_dict['targets']
        scores = train_elements_dict['scores']
        seasons = train_elements_dict['seasons']
        teams = train_elements_dict['teams']
        standings = train_elements_dict['standings']

        print('calc target stat ... ', end="")

        ## target stats
        targets_train = targets.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        targets_train = targets_train[(targets_train['date'] >= 20210401)]

        playerId_list = targets_train['playerId'].unique()
        player_target_probs = pd.DataFrame(columns = self.stat_cols)  
        for pid in tqdm(playerId_list):
            temp = targets_train[targets_train['playerId'] == pid]
            player_target_stats=calc_probs(pid,player_target_probs,temp)

        teamId_list = targets_train['teamId'].dropna().unique()
        team_target_probs = pd.DataFrame(columns = self.stat_cols)
        for pid in tqdm(teamId_list):
            temp = targets_train[targets_train['teamId'] == pid]
            team_target_stats=calc_probs(pid,team_target_probs,temp)

        team_stat_cols = self.stat_cols
        team_stat_cols = team_stat_cols[:1] + ["team_" + word for word in team_stat_cols[1:]]
        team_target_stats.columns = team_stat_cols

        self.feature_cols1 += self.stat_cols[1:-6]
        self.feature_cols2 += self.stat_cols[1:-6]
        self.feature_cols3 += self.stat_cols[1:-6]
        self.feature_cols4 += self.stat_cols[1:-6]

        self.feature_cols1 += team_stat_cols[1:]
        self.feature_cols2 += team_stat_cols[1:]
        self.feature_cols3 += team_stat_cols[1:]
        self.feature_cols4 += team_stat_cols[1:]

        print('done.')

        print('preprocess ... ', end="")
        ## salaries
        # salaries = salaries.groupby(['year', 'team']).sum()['salary'].reset_index()
        # salaries['team'] = salaries['team'].apply(map_team_name)
        # salaries = salaries.merge(teams, left_on='team', right_on='name', how='inner')
        # salaries = salaries.rename(columns={'id': 'teamId'})

        ## seasons
        seasons = seasons.fillna('0000-00-00')
        for c_ in seasons.columns[1:]:
            seasons[c_] = seasons[c_].str.replace('-', '').astype(int)

        ## players
        players['DOY'] = pd.to_datetime(players['DOB'], format="%Y-%m-%d").dt.year
        players['mlbDebutYear'] = pd.to_datetime(players['mlbDebutDate'], format="%Y-%m-%d").dt.year
        players['DebutAge'] = players['mlbDebutYear'] - players['DOY']
        players['mlbDebutDate'] = pd.to_numeric(players['mlbDebutDate'].str.replace('-', ''), errors="coerce")

        print('done.')

        print('creat feature ... ', end="")
        # creat feature
        train = targets[self.targets_cols].merge(players[self.players_cols], on=['playerId'], how='left')
        train = train.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        train = train.merge(scores[self.scores_cols], on=['playerId', 'date'], how='left')
        train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
        train = train.merge(standings[self.standings_cols], on=['teamId', 'date'], how='left')
        train = train.merge(team_target_stats, how='left', left_on=["teamId"],right_on=["playerId"], suffixes=('', 'team_'))
        
        date_ = pd.to_datetime(train['date'], format="%Y%m%d")
        train['annual_day'] = (date_ - pd.to_datetime(date_.dt.year, format="%Y")) /  timedelta(days=1)
        train['week_day'] = date_.dt.weekday
        train['month'] = date_.dt.month
        train['year'] = date_.dt.year
        train['mlbDebutDateflag'] = (train['mlbDebutDate'] == train['date']) * 1
        train['sincemlbDebutDateflag'] = (train['date'] >= train['mlbDebutDate']) * 1
        train['diffmlbDebutDateflag'] = (train['date'] - train['mlbDebutDate'])

        # label encoding
        player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
        position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
        birthCityn2num = {c: i for i, c in enumerate(train['birthCity'].unique())}
        teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
        status2num = {c: i for i, c in enumerate(train['status'].unique())}
        train['label_playerId'] = train['playerId'].map(player2num)
        train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
        train['label_birthCity'] = train['birthCity'].map(birthCityn2num)
        train['label_teamId'] = train['teamId'].map(teamid2num)
        train['label_status'] = train['status'].map(status2num)

        ## season_info
        on_preseason_idxes = extract_season(train['date'], seasons[['preSeasonStartDate', 'preSeasonEndDate']])
        on_season_idxes = extract_season(train['date'], seasons[['regularSeasonStartDate', 'regularSeasonEndDate']]) * 2
        on_postseason_idxes = extract_season(train['date'], seasons[['postSeasonStartDate', 'postSeasonEndDate']]) * 3

        special_days = seasons['lastDate1stHalf'].to_list() + seasons['allStarDate'].to_list() + seasons['firstDate2ndHalf'].to_list()
        special_idxes = 0
        for day in special_days:
            special_idxes += (train['date'] == day) * 4

        on_total_season_idxes = on_preseason_idxes
        on_total_season_idxes[on_season_idxes==2] = 2
        on_total_season_idxes[on_postseason_idxes==3] = 3
        on_total_season_idxes[special_idxes==4] = 4

        train['season_info'] = on_total_season_idxes

        ## only on season
        on_whole_idxes = extract_season(train['date'], seasons[['seasonStartDate', 'seasonEndDate']])
        train = train[on_whole_idxes == 1].reset_index(drop=True)

        # train = train.merge(playerTwitterFollowers, how='left', on=["playerId", 'date'])

        ## only test_player
        train = train[train['playerForTestSetAndFuturePreds']==True].reset_index(drop=True)

        print('done.')
        
        train_features_dict = {'players': players,
                                'player_target_stats': player_target_stats,
                                'team_target_stats': team_target_stats,
                                'player2num': player2num, 
                                'position2num': position2num, 
                                'birthCityn2num': birthCityn2num,
                                'teamid2num': teamid2num,
                                'status2num': status2num,
                                'feature_cols1': self.feature_cols1,
                                'feature_cols2': self.feature_cols2,
                                'feature_cols3': self.feature_cols3,
                                'feature_cols4': self.feature_cols4
                              }
        
        if self.usetimelinefeature:
            ## game_info
            train['gameday'] = ~train['battingOrder'].isna()*1
            train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

            train=count_consecutive_items_n_cols(train,['playerId','gameday'],'daysSinceLastGame')
            train.loc[train['gameday']==1,'daysSinceLastGame'] = 0

            train_game = train[train['gameday']==1]
            train_last_game = train_game[~train_game.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
            train_last_game.columns = ['playerId', 'lastdate']
            train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
            train_last_game = pd.merge(train_player_unique, train_last_game, on=['playerId'], how='left' )
            train_last_game = train_last_game.fillna(20171231)
            
            train_features_dict['train_last_game'] = train_last_game
            self.feature_cols1 += ['daysSinceLastGame']
            self.feature_cols2 += ['daysSinceLastGame']
            self.feature_cols3 += ['daysSinceLastGame']
            self.feature_cols4 += ['daysSinceLastGame']
            
            
            ## rosters_info
            train['rosterday'] = ~train['status'].isna()*1
            train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

            train=count_consecutive_items_n_cols(train,['playerId','rosterday'],'daysSinceLastRoster')
            train.loc[train['rosterday']==1,'daysSinceLastRoster'] = 0

            train_roster= train[train['rosterday']==1]
            train_last_roster = train_roster[~train_roster.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
            train_last_roster.columns = ['playerId', 'lastroster']
            train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
            train_last_roster = pd.merge(train_player_unique, train_last_roster, on=['playerId'], how='left' )
            train_last_roster = train_last_roster.fillna(20171231)
            
            train_features_dict['train_last_roster'] = train_last_roster
            self.feature_cols1 += ['daysSinceLastRoster']
            self.feature_cols2 += ['daysSinceLastRoster']
            self.feature_cols3 += ['daysSinceLastRoster']
            self.feature_cols4 += ['daysSinceLastRoster']


        return train, train_features_dict
    
    def train_and_evaluate(self, train, isgamedayonly=False):
        
        if isgamedayonly:
            train = train[train['gameday'] == 1].reset_index(drop=True)
            
        train_X = train
        train_y = train[['target1', 'target2', 'target3', 'target4']]

        oof = np.zeros(train_y.shape) - 1.0
        y_valids = np.zeros(train_y.shape) - 1.0

        tr_idx = (train['date'].astype(int) < 20210401)
        val_idx = ~tr_idx

        x_train = train_X.loc[tr_idx].reset_index(drop=True)
        y_train = train_y.loc[tr_idx].reset_index(drop=True)
        x_valid = train_X.loc[val_idx].reset_index(drop=True)
        y_valid = train_y.loc[val_idx].reset_index(drop=True)

        study1 = optuna.create_study(direction='maximize')
        study1.optimize(fit_lgbm(x_train[self.feature_cols1], y_train['target1'], 
                                 x_valid[self.feature_cols1], y_valid['target1']), n_trials=100)
        
        study2 = optuna.create_study(direction='maximize')
        study2.optimize(fit_lgbm(x_train[self.feature_cols2], y_train['target2'], 
                                 x_valid[self.feature_cols2], y_valid['target2']), n_trials=100)
        
        study3 = optuna.create_study(direction='maximize')
        study3.optimize(fit_lgbm(x_train[self.feature_cols3], y_train['target3'], 
                                 x_valid[self.feature_cols3], y_valid['target3']), n_trials=100)
        
        study4 = optuna.create_study(direction='maximize')
        study4.optimize(fit_lgbm(x_train[self.feature_cols4], y_train['target4'], 
                                 x_valid[self.feature_cols4], y_valid['target4']), n_trials=100)

#         oof1, model1, score1 = fit_lgbm(
#             x_train[self.feature_cols1], y_train['target1'],
#             x_valid[self.feature_cols1], y_valid['target1'],
# #             self.params1
#         )
#         oof2, model2, score2 = fit_lgbm(
#             x_train[self.feature_cols2], y_train['target2'],
#             x_valid[self.feature_cols2], y_valid['target2'],
# #             self.params2
#         )
#         oof3, model3, score3 = fit_lgbm(
#             x_train[self.feature_cols3], y_train['target3'],
#             x_valid[self.feature_cols3], y_valid['target3'],
# #             self.params3
#         )
#         oof4, model4, score4 = fit_lgbm(
#             x_train[self.feature_cols4], y_train['target4'],
#             x_valid[self.feature_cols4], y_valid['target4'],
# #             self.params4
#         )

#         score = (score1+score2+score3+score4) / 4
#         print(f'score: {score}')

#         oof[val_idx, 0] = oof1
#         oof[val_idx, 1] = oof2
#         oof[val_idx, 2] = oof3
#         oof[val_idx, 3] = oof4
#         y_valids[val_idx, 0] = y_valid['target1'].values
#         y_valids[val_idx, 1] = y_valid['target2'].values
#         y_valids[val_idx, 2] = y_valid['target3'].values
#         y_valids[val_idx, 3] = y_valid['target4'].values

#         mae = mean_absolute_error(y_valids[val_idx, :], oof[val_idx, :])
#         print("mae:", mae)

#         val_idx_num = val_idx[val_idx==True].index.to_list()

#         oof_df = train[self.targets_cols]
#         oof_df.iloc[val_idx_num, 1:5] = oof[val_idx_num, :]

#         models = np.array([model1, model2, model3, model4])

        return study1, study2, study3, study4
            

In [17]:
rt4kaido_train = Rt4kaidoTrain(usetimelinefeature=True)
train, train_features_dict = rt4kaido_train.make_feature(train_elements_dict)

calc target stat ... 

100%|██████████| 30/30 [00:00<00:00, 42.46it/s]


done.
preprocess ... done.
creat feature ... done.


In [18]:
study1, study2, study3, study4 = rt4kaido_train.train_and_evaluate(train, isgamedayonly=False)

[I 2021-07-18 15:51:27,101] A new study created in memory with name: no-name-3c45e163-3b14-4eb4-938d-73b78eb66008


[LightGBM] [Warning] feature_fraction is set=0.9799513504360726, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.9799513504360726
[LightGBM] [Warning] bagging_fraction is set=0.6832069489833392, subsample=0.6 will be ignored. Current value: bagging_fraction=0.6832069489833392
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.891911	valid_1's l1: 1.2236
[200]	training's l1: 0.891133	valid_1's l1: 1.22237
[300]	training's l1: 0.891129	valid_1's l1: 1.22237
[400]	training's l1: 0.891126	valid_1's l1: 1.22237
[500]	training's l1: 0.887146	valid_1's l1: 1.21481
[600]	training's l1: 0.885099	valid_1's l1: 1.21105
[700]	training's l1: 0.883507	valid_1's l1: 1.20827
[800]	training's l1: 0.88285	valid_1's l1: 1.20669
[900]	training's l1: 0.880264	valid_1's l1: 1.20035


[I 2021-07-18 15:51:59,342] Trial 0 finished with value: -1.2002791304146792 and parameters: {'max_depth': 17, 'min_child_weight': 9, 'subsample': 0.6, 'colsample_bytree': 0.9, 'reg_lambda': 26.422101105946066, 'reg_alpha': 0.029822905322017965, 'feature_fraction': 0.9799513504360726, 'bagging_fraction': 0.6832069489833392, 'bagging_freq': 2, 'num_leaves': 21, 'min_child_samples': 56}. Best is trial 0 with value: -1.2002791304146792.


Early stopping, best iteration is:
[824]	training's l1: 0.880326	valid_1's l1: 1.20032
[LightGBM] [Warning] feature_fraction is set=0.29985449619151255, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.29985449619151255
[LightGBM] [Warning] bagging_fraction is set=0.8676600442770488, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8676600442770488
[LightGBM] [Warning] bagging_freq is set=11, subsample_freq=0 will be ignored. Current value: bagging_freq=11
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.836087	valid_1's l1: 1.15726
[200]	training's l1: 0.822027	valid_1's l1: 1.15016


[I 2021-07-18 15:52:11,824] Trial 1 finished with value: -1.1495600977897178 and parameters: {'max_depth': 10, 'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_lambda': 102.46953401877913, 'reg_alpha': 0.74655927543029, 'feature_fraction': 0.29985449619151255, 'bagging_fraction': 0.8676600442770488, 'bagging_freq': 11, 'num_leaves': 763, 'min_child_samples': 25}. Best is trial 1 with value: -1.1495600977897178.


...

Early stopping, best iteration is:
[59]	training's l1: 0.819076	valid_1's l1: 1.56114
[LightGBM] [Warning] feature_fraction is set=0.8562158037482217, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.8562158037482217
[LightGBM] [Warning] bagging_fraction is set=0.380101128112707, subsample=0.6 will be ignored. Current value: bagging_fraction=0.380101128112707
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.880551	valid_1's l1: 1.57316


[I 2021-07-18 17:34:15,202] Trial 98 finished with value: -1.5648277697703363 and parameters: {'max_depth': 9, 'min_child_weight': 5, 'subsample': 0.6, 'colsample_bytree': 0.5, 'reg_lambda': 0.01967585260944778, 'reg_alpha': 0.24103188887876276, 'feature_fraction': 0.8562158037482217, 'bagging_fraction': 0.380101128112707, 'bagging_freq': 1, 'num_leaves': 677, 'min_child_samples': 31}. Best is trial 64 with value: -1.5408970069814598.


Early stopping, best iteration is:
[60]	training's l1: 0.91273	valid_1's l1: 1.56489
[LightGBM] [Warning] feature_fraction is set=0.7605152526599908, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.7605152526599908
[LightGBM] [Warning] bagging_fraction is set=0.9186267984686527, subsample=0.6 will be ignored. Current value: bagging_fraction=0.9186267984686527
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.77086	valid_1's l1: 1.57634


[I 2021-07-18 17:34:30,901] Trial 99 finished with value: -1.5566002756868231 and parameters: {'max_depth': 18, 'min_child_weight': 2, 'subsample': 0.6, 'colsample_bytree': 0.6, 'reg_lambda': 0.4011441470673972, 'reg_alpha': 0.5547087454603473, 'feature_fraction': 0.7605152526599908, 'bagging_fraction': 0.9186267984686527, 'bagging_freq': 4, 'num_leaves': 700, 'min_child_samples': 16}. Best is trial 64 with value: -1.5408970069814598.


Early stopping, best iteration is:
[57]	training's l1: 0.81356	valid_1's l1: 1.55671


In [19]:
print(f'best_score = {-study1.best_value}')
study1.best_trial.params

best_score = 1.1354083510221529


{'max_depth': 17,
 'min_child_weight': 7,
 'subsample': 0.9,
 'colsample_bytree': 0.6,
 'reg_lambda': 0.07269362050717991,
 'reg_alpha': 0.01180280012636004,
 'feature_fraction': 0.9659395841797106,
 'bagging_fraction': 0.735264203854535,
 'bagging_freq': 6,
 'num_leaves': 999,
 'min_child_samples': 12}

In [20]:
print(f'best_score = {-study2.best_value}')
study2.best_trial.params

best_score = 2.1928612683905775


{'max_depth': 20,
 'min_child_weight': 3,
 'subsample': 0.8,
 'colsample_bytree': 0.7,
 'reg_lambda': 0.12437298804990757,
 'reg_alpha': 0.7571420981596358,
 'feature_fraction': 0.9735850811591042,
 'bagging_fraction': 0.9191131896514854,
 'bagging_freq': 12,
 'num_leaves': 756,
 'min_child_samples': 27}

In [21]:
print(f'best_score = {-study3.best_value}')
study3.best_trial.params

best_score = 0.9247521521259133


{'max_depth': 10,
 'min_child_weight': 4,
 'subsample': 0.9,
 'colsample_bytree': 0.8,
 'reg_lambda': 0.0030321946484399885,
 'reg_alpha': 1.7393901648481276,
 'feature_fraction': 0.9471377517088052,
 'bagging_fraction': 0.5991586009727979,
 'bagging_freq': 16,
 'num_leaves': 654,
 'min_child_samples': 81}

In [22]:
print(f'best_score = {-study4.best_value}')
study4.best_trial.params

best_score = 1.5408970069814598


{'max_depth': 12,
 'min_child_weight': 8,
 'subsample': 0.6,
 'colsample_bytree': 0.6,
 'reg_lambda': 0.02614620036703161,
 'reg_alpha': 0.9696800599324547,
 'feature_fraction': 0.792727010920033,
 'bagging_fraction': 0.5620353986573813,
 'bagging_freq': 2,
 'num_leaves': 613,
 'min_child_samples': 42}